This is an example of SimSwap on processing video with multiple faces with designated sources.

Code path: https://github.com/lokesheywa/SimSwapLokesh
Paper path: https://arxiv.org/pdf/2106.06340v1.pdf.

In [ ]:
## make sure you are using a runtime with GPU
## you can check at Runtime/Change runtime type in the top bar.
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


All file changes make by this notebook are temporary.
You can try to mount your own google drive to store files if you wang.


In [ ]:
import os
%cd /content
!rm -rf SimSwap
!git clone https://github.com/lokesheywa/SimSwapLokesh SimSwap
!cd SimSwap && git pull

/content
Cloning into 'SimSwap'...
remote: Enumerating objects: 1295, done.
remote: Counting objects: 100% (567/567), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 1295 (delta 473), reused 426 (delta 426), pack-reused 728 (from 2)
Receiving objects: 100% (1295/1295), 211.53 MiB | 34.45 MiB/s, done.
Resolving deltas: 100% (703/703), done.
Already up to date.


In [ ]:
!pip install insightface==0.2.1 onnxruntime moviepy
!pip install googledrivedownloader
!pip install imageio==2.4.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303884 sha256=fbf76656110a60c5bc6b8a1e7fed780d89c012e2fe9c12539e77facf2b9b7c54
  Stored in directory: /root/.cache/pip/wheels/1b/28/50/248b15750b57c6b163d89d265f242e9cf6bce0bedfea3120aa
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.37.0
    Uninstalling imageio-2.37.0:
      Successfully uninstalled imageio-2.37.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [ ]:
import os
os.chdir("SimSwap")
!ls

 cog.yaml	       README.md
 crop_224	      'SimSwap colab.ipynb'
 data		       simswaplogo
 demo_file	       test_one_image.py
 docs		       test_video_swapmulti.py
 download-weights.sh   test_video_swap_multispecific.py
 insightface_func      test_video_swapsingle.py
 LICENSE	       test_video_swapspecific.py
 models		       test_wholeimage_swapmulti.py
 MultiSpecific.ipynb   test_wholeimage_swap_multispecific.py
 options	       test_wholeimage_swapsingle.py
 output		       test_wholeimage_swapspecific.py
 parsing_model	       train.ipynb
 pg_modules	       train.py
 predict.py	       util


In [ ]:
## You can upload filed manually
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:


### it seems that google drive link may not be permenant, you can find this ID from our open url.
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N',
#                                     dest_path='./arcface_model/arcface_checkpoint.tar')
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1PXkRiBUYbu1xWpQyDEJvGKeqqUFthJcI',
#                                     dest_path='./checkpoints.zip')

!wget -P ./arcface_model https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!unzip ./checkpoints.zip  -d ./checkpoints
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth

--2025-08-02 02:59:22--  https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/374891081/f01468b3-446b-4867-8c78-6d496183f9e6?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-02T03%3A53%3A50Z&rscd=attachment%3B+filename%3Darcface_checkpoint.tar&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-02T02%3A53%3A45Z&ske=2025-08-02T03%3A53%3A50Z&sks=b&skv=2018-11-09&sig=CIyBToCzkVMBu3UnN9dbGX%2FAJ8%2B49FFq%2BZYxi4Z0hYM%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NDEwMzg2MiwibmJmIjoxNzU0MTAzNTYyLCJwYXRoIjoicmVsZWFzZWFzc2V0cH

In [ ]:
!pip install gdown
!gdown 1tNRVUUoYiJraBDpoCoHIZbWaOZ1TF1Ln -O antelope.zip
!mkdir -p insightface_func/models/antelope
!unzip -q antelope.zip -d insightface_func/models/


Downloading...
From (original): https://drive.google.com/uc?id=1tNRVUUoYiJraBDpoCoHIZbWaOZ1TF1Ln
From (redirected): https://drive.google.com/uc?id=1tNRVUUoYiJraBDpoCoHIZbWaOZ1TF1Ln&confirm=t&uuid=8543f4c8-f301-46ad-9a5c-6caf97d43267
To: /content/SimSwap/antelope.zip
100% 248M/248M [00:05<00:00, 48.3MB/s]


In [ ]:
import cv2
import torch
import fractions
import numpy as np
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop
from util.videoswap_multispecific import video_swap
import os
import glob

  if event.key is 'enter':



In [ ]:
def lcm(a, b): return abs(a * b) / fractions.gcd(a, b) if a and b else 0

transformer = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

transformer_Arcface = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [ ]:
!ls ./checkpoints

people


In [ ]:
opt = TestOptions()
opt.initialize()
opt.parser.add_argument('-f') ## dummy arg to avoid bug
opt = opt.parse()
opt.multisepcific_dir = '/content/drive/MyDrive/imageProcessing/demo_file/multispecific' ## or replace it with folder from your own google drive
                           ## and remember to follow the dir structure in usage.md
opt.video_path = '/content/drive/MyDrive/imageProcessing/demo_file/multi_people_1080p.mp4' ## or replace it with video from your own google drive
opt.output_path = '/content/drive/MyDrive/multi_test_multispecific.mp4'
opt.temp_path = './tmp'
opt.Arc_path = './arcface_model/arcface_checkpoint.tar'
opt.name = 'people'
opt.isTrain = False
opt.use_mask = True  ## new feature up-to-date

crop_size = opt.crop_size


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
import models.arcface_models as arc
from models.arcface_models import ResNet,IRBlock,SEBlock
from torch.nn import Conv2d, BatchNorm2d, ReLU, Sequential, AdaptiveAvgPool2d, Linear ,PReLU ,MaxPool2d,Sigmoid,Dropout,BatchNorm1d
print(dir(arc))
torch.serialization.add_safe_globals({ResNet,Conv2d,BatchNorm2d, ReLU,Sequential, AdaptiveAvgPool2d, Linear,PReLU,MaxPool2d,IRBlock,SEBlock,Sigmoid,Dropout,BatchNorm1d})

import numpy as np

# Monkey patch deprecated numpy types
np.float = float
np.int = int
np.bool = bool
np.object = object
np.str = str

In [ ]:
pic_specific = opt.pic_specific_path
crop_size = opt.crop_size
multisepcific_dir = opt.multisepcific_dir

torch.nn.Module.dump_patches = True
model = create_model(opt)
model.eval()

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))
# The specific person to be swapped(source)
source_specific_id_nonorm_list = []
source_path = os.path.join(multisepcific_dir,'SRC_*')
source_specific_images_path = sorted(glob.glob(source_path))

with torch.no_grad():
    for source_specific_image_path in source_specific_images_path:
        specific_person_whole = cv2.imread(source_specific_image_path)
        specific_person_align_crop, _ = app.get(specific_person_whole,crop_size)
        specific_person_align_crop_pil = Image.fromarray(cv2.cvtColor(specific_person_align_crop[0],cv2.COLOR_BGR2RGB))
        specific_person = transformer_Arcface(specific_person_align_crop_pil)
        specific_person = specific_person.view(-1, specific_person.shape[0], specific_person.shape[1], specific_person.shape    [2])
        # convert numpy to tensor
        specific_person = specific_person.cuda()
        #create latent id
        specific_person_downsample = F.interpolate(specific_person, size=(112,112))
        specific_person_id_nonorm = model.netArc(specific_person_downsample)
        source_specific_id_nonorm_list.append(specific_person_id_nonorm.clone())

    # The person who provides id information (list)
    target_id_norm_list = []
    target_path = os.path.join(multisepcific_dir,'DST_*')
    target_images_path = sorted(glob.glob(target_path))

    for target_image_path in target_images_path:
        img_a_whole = cv2.imread(target_image_path)
        img_a_align_crop, _ = app.get(img_a_whole,crop_size)
        img_a_align_crop_pil = Image.fromarray(cv2.cvtColor(img_a_align_crop[0],cv2.COLOR_BGR2RGB))
        img_a = transformer_Arcface(img_a_align_crop_pil)
        img_id = img_a.view(-1, img_a.shape[0], img_a.shape[1], img_a.shape[2])
        # convert numpy to tensor
        img_id = img_id.cuda()
        #create latent id
        img_id_downsample = F.interpolate(img_id, size=(112,112))
        latend_id = model.netArc(img_id_downsample)
        latend_id = F.normalize(latend_id, p=2, dim=1)
        target_id_norm_list.append(latend_id.clone())

    assert len(target_id_norm_list) == len(source_specific_id_nonorm_list), "The number of images in source and target  directory must be same !!!"
    video_swap(opt.video_path, target_id_norm_list,source_specific_id_nonorm_list, opt.id_thres, \
        model, app, opt.output_path,temp_results_dir=opt.temp_path,no_simswaplogo=opt.no_simswaplogo,use_mask=opt.use_mask)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil

# Define paths
local_video_path = './output/multi_test_multispecific.mp4 '
drive_video_path = '/content/drive/MyDrive/uploaded_video.mp4'  # Change file name if needed

# Copy the video
shutil.copy(local_video_path, drive_video_path)
print(f'Video uploaded to: {drive_video_path}')